In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt

In [11]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 5)
        self.conv2 = nn.Conv2d(32, 64, 5)        
        self.conv3 = nn.Conv2d(64, 128, 5)        
        
        x = torch.randn(50, 50).view(-1, 1, 50, 50)
        self._to_linear = None
        self.convs(x)
        
        self.fc1 = nn.Linear(self._to_linear, 512)
        self.fc2 = nn.Linear(512, 2)
    
    def convs(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2, 2))
    
        if self._to_linear is None:
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
        return x
    
    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim=1)

In [12]:
data = np.load("training_data.npy", allow_pickle=True)

net = Net()

print(len(data))

24946


In [13]:
opt = torch.optim.Adam(net.parameters(), lr=0.001)
loss_fun = nn.MSELoss()

X = torch.Tensor([i[0] for i in data]).view(-1, 50, 50)
X = X / 255.0
y = torch.Tensor([i[1] for i in data])

In [14]:
VAL_CUT = 0.1
val = int(len(data)*VAL_CUT)

In [15]:
train_x = X[:-val]
train_y = y[:-val]

test_x = X[-val:]
test_y = y[-val:]

In [25]:
def train(net):
    opt = torch.optim.Adam(net.parameters(), lr=0.001)
    for epoch in range(1):
        for i in range(0, len(train_x), 100):
            batch_x = train_x[i:i+100].view(-1, 1, 50, 50)
            batch_y = train_y[i:i+100]
            
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            net.zero_grad()
            
            opt.zero_grad()
            outputs = net(batch_x)
            loss = loss_fun(outputs, batch_y)
            loss.backward()
            opt.step()
        print(f"Epoch {epoch}. Loss: {loss}")

In [29]:
def test(net):
    correct = 0
    total = 0
    
    with torch.no_grad():
        for i in range(len(test_x)):
            real_class = torch.argmax(test_y[i]).to(device)
            net_out = net(test_x[i].view(-1, 1, 50, 50).to(device))[0]
            predicted_class = torch.argmax(net_out)
            
            if predicted_class == real_class:
                correct += 1
            total += 1
        print(f"Accuracy {round(correct/total, 3)}")

In [19]:
device = torch.device("cuda:0")
device

device(type='cuda', index=0)

In [20]:
torch.cuda.device_count()

1

In [21]:
net.to(device)

Net(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=512, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=2, bias=True)
)

In [26]:
train(net)

Epoch 0. Loss: 0.2441929578781128


In [30]:
test(net)

Accuracy 0.605
